# Test de Migration - Nouvelle Structure Modulaire (Version Fixée)

Ce notebook teste la nouvelle structure modulaire du projet STA211 avec gestion automatique des chemins.

## 0. Configuration des chemins d'import

In [ ]:
import sys
import os
from pathlib import Path

# Détection automatique de la racine du projet
def find_project_root():
    """Trouve la racine du projet en cherchant les marqueurs."""
    current_dir = Path.cwd()
    
    # Si on exécute depuis notebooks/, remonter d'un niveau
    if current_dir.name == 'notebooks':
        project_root = current_dir.parent
    else:
        # Chercher les marqueurs dans les répertoires parents
        for parent in [current_dir] + list(current_dir.parents):
            markers = ['.git', '.project_root', 'modules', 'requirements.txt']
            if any((parent / marker).exists() for marker in markers):
                project_root = parent
                break
        else:
            project_root = current_dir
    
    return project_root

# Trouver et ajouter la racine du projet au PATH
project_root = find_project_root()
print(f"📁 Racine du projet détectée: {project_root}")

# Ajouter au sys.path si pas déjà présent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))
    print(f"✅ Ajouté au sys.path: {project_root}")
else:
    print(f"ℹ️ Déjà dans sys.path: {project_root}")

# Vérifier que le répertoire modules existe
modules_dir = project_root / 'modules'
if modules_dir.exists():
    print(f"✅ Répertoire modules trouvé: {modules_dir}")
else:
    print(f"❌ Répertoire modules non trouvé: {modules_dir}")
    print("Contenu du répertoire racine:")
    for item in project_root.iterdir():
        print(f"  - {item.name}")

## 1. Test de la configuration

In [ ]:
try:
    from modules.config import cfg
    print("✅ Configuration importée avec succès")
    
    print(f"📁 Racine du projet: {cfg.root}")
    print(f"🏷️ Nom du projet: {cfg.project.project_name}")
    print(f"👤 Auteur: {cfg.project.author}")
    print(f"📊 Random state: {cfg.project.random_state}")

    print("\n📂 Chemins configurés:")
    for attr_name in ['data', 'raw', 'processed', 'outputs', 'figures', 'models', 'imputers', 'transformers']:
        try:
            path = getattr(cfg.paths, attr_name)
            exists = "✅" if path.exists() else "❌"
            print(f"  {exists} {attr_name}: {path}")
        except AttributeError:
            print(f"  ⚠️ {attr_name}: attribut non trouvé")
            
except ImportError as e:
    print(f"❌ Erreur d'import de la configuration: {e}")
    print(f"Répertoire de travail actuel: {Path.cwd()}")
    print(f"sys.path contient: {sys.path[:3]}...")  # Premier 3 éléments

## 2. Test des utilitaires de stockage

In [ ]:
try:
    from modules.utils import save_artifact, load_artifact, artifact_exists
    import numpy as np

    print("✅ Utilitaires de stockage importés")

    # Test de sauvegarde/rechargement
    test_data = {'test': 'data', 'array': np.array([1, 2, 3])}
    test_filename = "test_artifact.pkl"

    # Sauvegarder
    saved_path = save_artifact(test_data, test_filename, cfg.paths.models)
    print(f"💾 Données sauvegardées: {saved_path}")

    # Vérifier existence
    exists = artifact_exists(test_filename, cfg.paths.models)
    print(f"🔍 Artefact existe: {exists}")

    # Recharger
    loaded_data = load_artifact(test_filename, cfg.paths.models)
    print(f"📂 Données rechargées: {loaded_data}")

    # Vérifier intégrité
    integrity_check = (
        loaded_data['test'] == test_data['test'] and 
        np.array_equal(loaded_data['array'], test_data['array'])
    )
    print(f"✅ Intégrité des données: {integrity_check}")
    
except ImportError as e:
    print(f"❌ Erreur d'import des utilitaires: {e}")
except Exception as e:
    print(f"❌ Erreur lors du test: {e}")

## 3. Test du module de prétraitement

In [ ]:
try:
    from modules.notebook1_preprocessing import (
        load_and_clean_data,
        perform_knn_imputation,
        apply_optimal_transformations,
        detect_and_cap_outliers
    )
    import pandas as pd
    import numpy as np

    print("✅ Module de prétraitement importé")
    print("📦 Fonctions disponibles:")
    print("  - load_and_clean_data")
    print("  - perform_knn_imputation")
    print("  - apply_optimal_transformations")
    print("  - detect_and_cap_outliers")

    # Test avec des données factices
    np.random.seed(42)

    # Créer des données de test
    n_samples = 100
    test_df = pd.DataFrame({
        'feature1': np.random.normal(0, 1, n_samples),
        'feature2': np.random.normal(5, 2, n_samples),
        'target': np.random.binomial(1, 0.3, n_samples)
    })

    # Introduire quelques valeurs manquantes
    missing_idx = np.random.choice(n_samples, 10, replace=False)
    test_df.loc[missing_idx, 'feature1'] = np.nan

    print(f"\n📊 Données de test créées: {test_df.shape}")
    print(f"🕳️ Valeurs manquantes: {test_df.isnull().sum().sum()}")

    # Test KNN imputation
    df_imputed, imputer = perform_knn_imputation(
        test_df, ['feature1'], n_neighbors=5, save_imputer=False
    )

    remaining_missing = df_imputed.isnull().sum().sum()
    print(f"✅ Imputation KNN: {remaining_missing} valeurs manquantes restantes")
    
except ImportError as e:
    print(f"❌ Erreur d'import du module de prétraitement: {e}")
    # Diagnostics supplémentaires
    modules_path = project_root / 'modules'
    if modules_path.exists():
        print(f"Contenu du dossier modules:")
        for item in modules_path.iterdir():
            print(f"  - {item.name}")
except Exception as e:
    print(f"❌ Erreur lors du test: {e}")

## 4. Test des imports simples

In [ ]:
# Test basique des imports un par un pour diagnostiquer
print("🔍 DIAGNOSTIC DES IMPORTS")
print("=" * 40)

modules_to_test = [
    'modules',
    'modules.config',
    'modules.utils',
    'modules.utils.storage',
    'modules.evaluation',
    'modules.modeling',
    'modules.notebook1_preprocessing',
    'modules.notebook2_modeling'
]

for module_name in modules_to_test:
    try:
        __import__(module_name)
        print(f"✅ {module_name}")
    except ImportError as e:
        print(f"❌ {module_name}: {e}")
    except Exception as e:
        print(f"⚠️ {module_name}: {e}")

print(f"\n📍 Répertoire de travail: {Path.cwd()}")
print(f"📍 Racine du projet: {project_root}")
print(f"📍 sys.path[0]: {sys.path[0]}")

## 5. Vérification de l'environnement

In [ ]:
print("🔍 VÉRIFICATION DE L'ENVIRONNEMENT")
print("=" * 50)

# Vérifier les dépendances principales
dependencies = [
    'pandas', 'numpy', 'sklearn', 'matplotlib', 'seaborn', 
    'xgboost', 'joblib', 'scipy'
]

print("📦 Dépendances:")
for dep in dependencies:
    try:
        __import__(dep)
        print(f"  ✅ {dep}")
    except ImportError:
        print(f"  ❌ {dep}")

# Vérifier les dépendances optionnelles
optional_deps = ['chardet', 'missingno', 'ydata_profiling']
print("\n📦 Dépendances optionnelles:")
for dep in optional_deps:
    try:
        __import__(dep)
        print(f"  ✅ {dep}")
    except ImportError:
        print(f"  ⚠️ {dep} (optionnel)")

print(f"\n🐍 Version Python: {sys.version}")
print(f"💻 Plateforme: {os.name}")

# Instructions d'installation si nécessaire
missing_deps = []
for dep in dependencies:
    try:
        __import__(dep)
    except ImportError:
        missing_deps.append(dep)

if missing_deps:
    print(f"\n⚠️ Dépendances manquantes: {', '.join(missing_deps)}")
    print("\n💡 Pour installer les dépendances:")
    print("   pip install -r requirements.txt")
    print("   ou")
    print(f"   pip install {' '.join(missing_deps)}")
else:
    print("\n🎉 Toutes les dépendances principales sont installées !")

## Instructions de dépannage

Si vous rencontrez encore des problèmes d'import :

### 1. Vérifier la structure du projet
```
sta211-project/
├── modules/
│   ├── config.py
│   ├── utils/
│   ├── evaluation/
│   └── ...
└── notebooks/
    └── 00_Test_Migration_Fixed.ipynb
```

### 2. Installer les dépendances
```bash
pip install -r requirements.txt
```

### 3. Exécuter depuis le bon répertoire
- Si vous utilisez Jupyter Notebook, démarrez-le depuis la racine du projet
- Si vous utilisez JupyterLab ou VS Code, ouvrez le dossier racine du projet

### 4. Import manuel alternatif
Si les imports automatiques échouent, vous pouvez utiliser :

```python
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent))  # Depuis notebooks/
# ou
sys.path.insert(0, r'C:\chemin\vers\sta211-project')  # Chemin absolu
```